# Machine Learning Foundation

## Section 2, Part c: Cross Validation 


## Learning objectives

By the end of this lesson, you will be able to:

* Chain multiple data processing steps together using `Pipeline`
* Use the `KFolds` object to split data into multiple folds.
* Perform cross validation using SciKit Learn with `cross_val_predict` and `GridSearchCV`


In [ ]:
# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline

In [ ]:
# Note we are loading a slightly different ("cleaned") pickle file
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML240EN-SkillsNetwork/labs/data/boston_housing_clean.pickle
# boston = pickle.load(open('data/boston_housing_clean.pickle', "rb" ))
boston = pickle.load(open('boston_housing_clean.pickle', "rb" ))

In [ ]:
boston.keys()

In [ ]:
!python --version

In [ ]:
boston_data = boston['dataframe']
boston_description = boston['description']

In [ ]:
boston_data.head()

### Discussion: 

Suppose we want to do Linear Regression on our dataset to get an estimate, based on mean squared error, of how well our model will perform on data outside our dataset. 

Suppose also that our data is split into three folds: Fold 1, Fold 2, and Fold 3.

What would the steps be, in English, to do this?


#### Coding this up

The [`KFold`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) object in SciKit Learn tells the cross validation object (see below) how to split up the data:


In [ ]:
X = boston_data.drop('MEDV', axis=1)
y = boston_data.MEDV

In [ ]:
kf = KFold(shuffle=True, random_state=72018, n_splits=3)

In [ ]:
for train_index, test_index in kf.split(X):
    print("Train index:", train_index[:10], len(train_index))
    print("Test index:",test_index[:10], len(test_index))
    print('')

In [ ]:
#from sklearn.metrics import r2_score, mean_squared_error

scores = []
lr = LinearRegression()

for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = (X.iloc[train_index, :], 
                                        X.iloc[test_index, :], 
                                        y[train_index], 
                                        y[test_index])
    
    lr.fit(X_train, y_train)
        
    y_pred = lr.predict(X_test)

    score = r2_score(y_test.values, y_pred)
    
    scores.append(score)
    
scores

A bit cumbersome, but do-able.


### **Resumen del texto:**
Este cuaderno pertenece al segundo curso sobre validación cruzada y se enfoca en la **canalización de procesamiento de datos** para optimizar flujos de trabajo en **Machine Learning**. Se cubren los siguientes temas clave:

- Uso de **KFolds** para dividir datos en múltiples pliegues y realizar validación cruzada.
- Implementación de **cross_val_predict** y **GridSearchCV** para evaluar el rendimiento del modelo.
- Introducción a la **regresión lineal**, junto con técnicas de **Lasso y Ridge** para evitar sobreajuste.
- Uso de **Pickle** para cargar y manejar datos en formato de diccionario.
- Aplicación práctica en el conjunto de datos **Boston Housing**, donde el objetivo es predecir el valor medio de viviendas usando diferentes características.
- Implementación de la validación cruzada mediante **tres pliegues**, asegurando que los conjuntos de prueba sean exclusivos.
- Uso de **r2_score** para medir el rendimiento del modelo en cada iteración.
- Destacar la importancia de realizar múltiples pliegues y promediar los resultados para mejorar la generalización del modelo.
- Próximo paso: agregar escalado y aplicar predicción cruzada con **cross_val_predict**.

Este cuaderno proporciona una base sólida para aplicar validación cruzada de manera eficiente en modelos de Machine Learning.

### Discussion (Part 2): 

Now suppose we want to do the same, but appropriately scaling our data as we go through the folds.

What would the steps be _now_?


### Coding this up


In [ ]:
scores = []

lr = LinearRegression()
s = StandardScaler()

for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = (X.iloc[train_index, :], 
                                        X.iloc[test_index, :], 
                                        y[train_index], 
                                        y[test_index])
    
    X_train_s = s.fit_transform(X_train)
    
    lr.fit(X_train_s, y_train)
    
    X_test_s = s.transform(X_test)
    
    y_pred = lr.predict(X_test_s)

    score = r2_score(y_test.values, y_pred)
    
    scores.append(score)

In [ ]:
scores

(same scores, because for vanilla linear regression with no regularization, scaling actually doesn't matter for performance)


This is getting quite cumbersome! 

_Very_ luckily, SciKit Learn has some wonderful functions that handle a lot of this for us.


### `Pipeline` and `cross_val_predict`


`Pipeline` lets you chain together multiple operators on your data that both have a `fit` method.


In [ ]:
s = StandardScaler()
lr = LinearRegression()

### Combine multiple processing steps into a `Pipeline`

A pipeline contains a series of steps, where a step is ("name of step", actual_model). The "name of step" string is only used to help you identify which step you are on, and to allow you to specify parameters at that step.  


In [ ]:
estimator = Pipeline([("scaler", s),
                      ("regression", lr)])

### `cross_val_predict`

[`cross_val_predict`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) is a function that does K-fold cross validation for us, appropriately fitting and transforming at every step of the way.


In [ ]:
kf

In [ ]:
predictions = cross_val_predict(estimator, X, y, cv=kf)

In [ ]:
r2_score(y, predictions)

In [ ]:
np.mean(scores) # almost identical!

Note that `cross_val_predict` doesn't use the same model for all steps; the predictions for each row are made when that row is in the validation set. We really have the collected results of 3 (i.e. `kf.num_splits`) different models. 

When we are done, `estimator` is still not fitted. If we want to predict on _new_ data, we still have to train our `estimator`. 


### **Resumen del texto:**
En esta segunda parte del cuaderno, se introduce el **preprocesamiento de datos**, la **canalización (pipeline)** y la **predicción cruzada (cross_val_predict)**, con el objetivo de optimizar el flujo de trabajo en Machine Learning.

#### **Temas principales:**
- **Escalado de datos** con **StandardScaler**, normalizando los valores para mejorar la estabilidad del modelo.
- **Automatización con pipelines** de `sklearn.pipeline`, permitiendo encadenar varios pasos como el escalado y la regresión lineal.
- **Uso de `cross_val_predict`**, que divide los datos en pliegues de entrenamiento y prueba sin necesidad de un bucle manual.
- Se demuestra que **escalar los datos no afecta el rendimiento** en una regresión lineal sin regularización, pero es esencial para técnicas como **Lasso y Ridge**.
- Explicación del proceso de **validación cruzada**, donde:
  - Los datos se dividen en **tres pliegues**.
  - Se entrena el modelo en **dos tercios** y se prueba en el **tercio restante**.
  - Se repite el proceso para cubrir todo el conjunto de datos.
  - Se obtiene el **r2_score** para evaluar el desempeño del modelo.
- Se aclara que **cross_val_predict no ajusta un solo modelo**, sino que entrena múltiples modelos en distintos subconjuntos de datos.

#### **Próximos pasos:**
- En la siguiente sección, se abordará el **ajuste de hiperparámetros**, que no son aprendidos directamente por el modelo, sino que deben ser configurados manualmente para mejorar el rendimiento.

Este cuaderno proporciona una **forma más eficiente de entrenar y validar modelos de Machine Learning** mediante la automatización de tareas repetitivas.

## Hyperparameter tuning


### Definition

**Hyperparameter tuning** involves using cross validation (or train-test split) to determine which hyperparameters are most likely to generate a model that _generalizes_ well outside of your sample.

### Mechanics

We can generate an exponentially spaces range of values using the numpy [`geomspace`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.geomspace.html#numpy.geomspace) function.

```python
np.geomspace(1, 1000, num=4)
```

produces:

```
array([    1.,    10.,   100.,  1000.])
```

Use this function to generate a list of length 10 called `alphas` for hyperparameter tuning:


In [ ]:
alphas = np.geomspace(1e-9, 1e0, num=10)
alphas

The code below tunes the `alpha` hyperparameter for Lasso regression.


In [ ]:
scores = []
coefs = []
for alpha in alphas:
    las = Lasso(alpha=alpha, max_iter=100000)
    
    estimator = Pipeline([
        ("scaler", s),
        ("lasso_regression", las)])

    predictions = cross_val_predict(estimator, X, y, cv = kf)
    
    score = r2_score(y, predictions)
    
    scores.append(score)

In [ ]:
list(zip(alphas,scores))

In [ ]:
Lasso(alpha=1e-6).fit(X, y).coef_

In [ ]:
Lasso(alpha=1.0).fit(X, y).coef_

In [ ]:
plt.figure(figsize=(10,6))
plt.semilogx(alphas, scores, '-o')
plt.xlabel('$\\alpha$')
plt.ylabel('$R^2$');

### Exercise

Add `PolynomialFeatures` to this `Pipeline`, and re-run the cross validation with the `PolynomialFeatures` added.

**Hint #1:** pipelines process input from first to last. Think about the order that it would make sense to add Polynomial Features to the data in sequence and add them in the appropriate place in the pipeline.

**Hint #2:** you should see a significant increase in cross validation accuracy from doing this


In [ ]:
pf = PolynomialFeatures(degree=2)
scores = []

alphas = np.geomspace(0.001, 10, 5)
for alpha in alphas:
    las = Lasso(alpha=alpha, max_iter=100000)
    estimator = Pipeline([
        ("make_higher_degree", pf),
        ("scaler", s),
        ("lasso_regression", las)])
    predictions = cross_val_predict(estimator, X, y, cv = kf)
    score = r2_score(y, predictions)
    
    scores.append(score)
scores

If you store the results in a list called `scores`, the following will work:


In [ ]:
plt.semilogx(alphas, scores);

In [ ]:
# Once we have found the hyperparameter (alpha~1e-2=0.01)
# make the model and train it on ALL the data
# Then release it into the wild .....
best_estimator = Pipeline([
                    ("make_higher_degree", PolynomialFeatures(degree=2)),
                    ("scaler", s),
                    ("lasso_regression", Lasso(alpha=0.01, max_iter=10000))])

best_estimator.fit(X, y)
best_estimator.score(X, y)

In [ ]:
best_estimator.named_steps["lasso_regression"].coef_

### **Resumen del texto:**
Esta sección del cuaderno se enfoca en el **ajuste de hiperparámetros**, particularmente en el uso de **regresión de Lasso** para regularizar modelos y mejorar su capacidad de generalización.

#### **Puntos clave:**
- **Diferencia entre hiperparámetros y parámetros:**  
  - **Parámetros** → Aprendidos automáticamente por el modelo.  
  - **Hiperparámetros** → Configurados manualmente para optimizar el rendimiento.  
- **Propósito del ajuste de hiperparámetros:**  
  - Encontrar el equilibrio entre **complejidad y error** mediante validación cruzada.
  - Identificar los hiperparámetros óptimos que generalicen bien a datos nuevos.
- **Uso de `np.geomspace` para definir valores de alfa:**  
  - Genera una secuencia de valores espaciados geométricamente para probar distintos niveles de regularización.
  - Valores más bajos → Modelos más complejos.  
  - Valores más altos → Modelos más simples.  
- **Implementación de regresión de Lasso con `Pipeline`:**  
  - Se escalán los datos (`StandardScaler`), luego se entrena la regresión (`Lasso`).
  - Se utiliza **validación cruzada (`cross_val_predict`)** para evaluar el desempeño en cada valor de alfa.
  - Se selecciona el **alfa óptimo**, que equilibra precisión y generalización.
- **Impacto de alfa en los coeficientes:**  
  - **Alfa bajo** → Modelos más complejos con más coeficientes activos.  
  - **Alfa alto** → Modelos más simples con coeficientes reducidos a cero (eliminación de variables irrelevantes).  
- **Expansión del modelo con características polinomiales:**  
  - Se agregan términos de interacción entre variables para mejorar el poder predictivo.
  - **El orden importa:** Primero se crean características polinomiales y luego se escalan.
- **Optimización de iteraciones máximas en Lasso:**  
  - Se aumenta `max_iter` para garantizar la convergencia del modelo.
- **Selección del mejor modelo:**  
  - Se identifica el **mejor valor de alfa** basado en su rendimiento en validación cruzada.
  - Se ajusta el modelo final con ese alfa y se evalúan los coeficientes eliminados por regularización.

#### **Próximos pasos:**
- **Explorar la regresión de Ridge**, que funciona de manera similar pero con una penalización diferente.
- **Comparar modelos** para encontrar la mejor estrategia de regularización.

Este cuaderno proporciona una guía estructurada para **ajustar modelos de Machine Learning** de manera eficiente y mejorar su capacidad de generalización a nuevos datos.

### Exercise

Do the same, but with `Ridge` regression 

Which model, `Ridge` or `Lasso`, performs best with its optimal hyperparameters on the Boston dataset?


In [ ]:
pf = PolynomialFeatures(degree=2)
scores = []

alphas = np.geomspace(4, 20, 20)
for alpha in alphas:
    ridge = Ridge(alpha=alpha, max_iter=100000)

    estimator = Pipeline([
        ("polynomial_features", pf),
        ("scaler", s),
        ("ridge_regression", ridge)])

    predictions = cross_val_predict(estimator, X, y, cv = kf)
    score = r2_score(y, predictions)
    scores.append(score)
plt.plot(alphas, scores)

**Conclusion:** Both Lasso and Ridge with proper hyperparameter tuning give better results than plain ol' Linear Regression!


### Exercise:


Now, for whatever your best overall hyperparameter was: 

* Standardize the data
* Fit and predict on the entire dataset
* See what the largest coefficients were
    * Hint: use 
    ```python
    dict(zip(model.coef_, pf.get_feature_names()))
    ```
    for your model `model` to get the feature names from `PolynomialFeatures`.
    
    Then, use
    ```python
    dict(zip(list(range(len(X.columns.values))), X.columns.values))
    ```
    
    to see which features in the `PolynomialFeatures` DataFrame correspond to which columns in the original DataFrame.


In [ ]:
# Once we have found the hyperparameter (alpha~1e-2=0.01)
# make the model and train it on ALL the data
# Then release it into the wild .....
best_estimator = Pipeline([
                    ("make_higher_degree", PolynomialFeatures(degree=2, include_bias=False)),
                    ("scaler", s),
                    ("lasso_regression", Lasso(alpha=0.01, max_iter=10000))])

best_estimator.fit(X, y)
best_estimator.score(X, y)

In [ ]:
df_importances = pd.DataFrame(zip(best_estimator.named_steps["make_higher_degree"].get_feature_names(),
                 best_estimator.named_steps["lasso_regression"].coef_,
))

In [ ]:
col_names_dict = dict(zip(list(range(len(X.columns.values))), X.columns.values))

In [ ]:
col_names_dict

In [ ]:
df_importances.sort_values(by=1)

## Grid Search CV


To do cross-validation, we used two techniques:
- use `KFolds` and manually create a loop to do cross-validation
- use `cross_val_predict` and `score` to get a cross-valiated score in a couple of lines.

To do hyper-parameter tuning, we see a general pattern:
- use `cross_val_predict` and `score` in a manually written loop over hyperparemeters, then select the best one.

Perhaps not surprisingly, there is a function that does this for us -- `GridSearchCV`


In [ ]:
from sklearn.model_selection import GridSearchCV

# Same estimator as before
estimator = Pipeline([("polynomial_features", PolynomialFeatures()),
        ("scaler", StandardScaler()),
        ("ridge_regression", Ridge())])

params = {
    'polynomial_features__degree': [1, 2, 3],
    'ridge_regression__alpha': np.geomspace(4, 20, 20)
}

grid = GridSearchCV(estimator, params, cv=kf)

In [ ]:
grid.fit(X, y)

In [ ]:
grid.best_score_, grid.best_params_

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (StandardScaler, 
                                   PolynomialFeatures)
from scipy.stats.mstats import normaltest
from scipy.stats import boxcox
from scipy.special import inv_boxcox

file_name='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ST0151EN-SkillsNetwork/labs/boston_housing.csv'
boston_data = pd.read_csv(file_name)

lr = LinearRegression()
y_col = "MEDV"
X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

pf = PolynomialFeatures(degree=2, include_bias=False)
X_pf = pf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_pf, y, test_size=0.3, 
                                                    random_state=72018)

s = StandardScaler()
X_train_s = s.fit_transform(X_train)

bc_result = boxcox(y_train)
y_train_bc = bc_result[0]
lam = bc_result[1]

lr.fit(X_train_s, y_train_bc)
X_test_s = s.transform(X_test)
y_pred_bc = lr.predict(X_test_s)

y_pred_tran = inv_boxcox(y_pred_bc, lam)
print(r2_score(y_pred_tran,y_test)) #RES 0.848052537981275

lr = LinearRegression()
lr.fit(X_train_s,y_train)
lr_pred = lr.predict(X_test_s)
r2_score(lr_pred,y_test) #RES 0.8667029116056716


In [ ]:
y_predict = grid.predict(X)

In [ ]:
# This includes both in-sample and out-of-sample
r2_score(y, y_predict)

In [ ]:
# Notice that "grid" is a fit object!
# We can use grid.predict(X_test) to get brand new predictions!
grid.best_estimator_.named_steps['ridge_regression'].coef_

In [ ]:
grid.cv_results_

## Summary

1. We can manually generate folds by using `KFolds`
2. We can get a score using `cross_val_predict(X, y, cv=KFoldObject_or_integer)`. 
   This will produce the out-of-bag prediction for each row.
3. When doing hyperparameter selection, we should be optimizing on out-of-bag scores. This means either using `cross_val_predict` in a loop, or ....
4. .... use `GridSearchCV`. GridSearchCV takes a model (or pipeline) and a dictionary of parameters to scan over. It finds the hyperparameter set that has the best out-of-sample score on all the parameters, and calls that it's "best estimator". It then retrains on all data with the "best" hyper-parameters.

### Extensions

Here are some additional items to keep in mind:
* There is a `RandomSearchCV` that tries random combination of model parameters. This can be helpful if you have a prohibitive number of combinations to test them all exhaustively.
* KFolds will randomly select rows to be in the training and test folds. There are other methods (such as `StratifiedKFolds` and `GroupKFold`, which are useful when you need more control over how the data is split (e.g. to prevent data leakage). You can create these specialized objects and pass them to the `cv` argument of `GridSearchCV`.
